#### This sample illustrtates how to use VALI for decoding and preprocessing together with YOLO

In [ ]:
from ultralytics import YOLO
import python_vali as vali
import torch

In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        return []

In [ ]:
model = YOLO("yolo11n.pt")
url = "../tests/data/test.mp4"
num_frames = 16
gpu_id = 0

In [ ]:
# HW-accelerated decoder.
pyDec = vali.PyDecoder(url, opts={}, gpu_id=gpu_id)

# Resize + color conversion combined operation.
# Use the same CUDA stream as decoder to avoid excessive CUDA syncs.
pyUD = vali.PySurfaceUD(gpu_id=gpu_id, stream=pyDec.Stream)

In [ ]:
target_w = 640
target_h = 480

# Pre-allocate GPU memory for decoded video frame and RGB tensor.
# Further operations on vali side will be done in-place.
surfaces = [
    vali.Surface.Make(
        format=pyDec.Format,
        width=pyDec.Width,
        height=pyDec.Height,
        gpu_id=gpu_id),

    vali.Surface.Make(
        format=vali.PixelFormat.RGB_32F_PLANAR,
        width=target_w,
        height=target_h,
        gpu_id=gpu_id)
]

In [ ]:
def decode_to_tensor() -> torch.tensor:
    """
    This function decodes single video frame and exports it
    to torch cuda tensor.

    Returns:
        torch.tensor: Planar RGB CUDA float tensor normalized to
        model liking.
    """
    # Decode single Surface.
    # Use async API because both classes share same CUDA stream.
    success, details = pyDec.DecodeSingleSurfaceAsync(surfaces[0])
    if not success:
        print(details)
        raise StopExecution
    
    # Convert to planar float RGB and resize in single shot.
    # Can't use async API because torch doesn't allow to construct
    # torch.cuda.Stream from handle.
    success, details = pyUD.Run(surfaces[0], surfaces[-1])
    if not success:
        print(details)
        raise StopExecution

    # Create torch tensor sharing actual memory with vali.
    img_tensor = torch.from_dlpack(surfaces[-1]).clamp(0.0, 1.0)
    img_tensor = torch.reshape(img_tensor, [1, 3, target_h, target_w])

    return img_tensor


In [ ]:
for i in range(0, num_frames):
    results = model.track(decode_to_tensor(), persist=True)
    print(results[0].to_json())